In [20]:
import graphics
import pomdp
from simulator import SimObject, Simulator
import tests
from tests import heuristic

import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Grasping using POMDPs

## Overview

When a robot is attempting to grasp an object, there is often uncertainty in its belief about where the object is located. To solve the problem of grasping under uncertainty, a robot can use <a href="https://en.wikipedia.org/wiki/Partially_observable_Markov_decision_process">Partially Observable Markov Decision Processes (POMDPs)</a> to choose optimal grasping actions based on its belief about the object's location and update its beliefs based on the actions it takes and observations it makes. If you'd like to learn more about grasping using POMDPs, you are welcome to read the article "Robot Grasping Under Object Pose Uncertainty" (Hsaio, Kaelbling, Perez) (included as a pdf).  

In this problem set, you will be implementing some of the functions used to model POMDPs. You will then be able to test your functions on a demo of a 2D robotic gripper grasping an object in a 2D world.

In this problem set, the world is represented as a 2D grid. Within the world, the location of the object is represented as its top left `(x, y)` coordinate. `(0, 0)` is the top left corner of the world. The `+x` direction is to the right, and the `+y` direction is down. 

This is an example of a `4x4` world with an object located at `(1, 1)`.

<img src="world_with_obj.png" style="width: 300px;">

The gripper is an object with two fingers. It has a touch sensor on the end of each of its fingers and one in the middle (see image below). The status of its touch sensors is represented as a list of booleans indicating whether each sensor is tangent to an object. For example, `[True, False, True]` would indicate that the sensors on the ends of its fingers are tangent to an object, but the middle one isn't. The sensor is grasping an object any time the middle sensor is tangent to an object.

<img src="gripper.png" style="width: 400px;">


The possible actions in this world are represented as all ways the gripper can move into the world. Actions are represented as a tuple of (`index`, `direction`), where `index` represents which row or column the gripper is centered at, and `direction` is one of `[TOP, LEFT, RIGHT, BOTTOM]`. The arrows in the image below represent all possible actions for that world. The red arrow represents the action `(1, TOP)`.  During the execution of an action, the gripper moves straight across the board, stopping if it collides with the object.

<img src="actions.png" style="width: 400px;">

## Simulation

Two Python classes have been provided for you to model this world: `Simulator` and `SimObject`.

### SimObject

`SimObject` represents an object to be grasped. We represent this object as a 2D [Numpy array](https://docs.scipy.org/doc/numpy/user/quickstart.html#array-creation) of 1s and 0s, with 1s representing cells occupied by the object and 0s representing cells not occupied by the object. The object should be contiguous and the bounds of the occupancy array should be tight; in other words, there should be no rows or columns containing only 0s. **NOTE: the occupancy array is indexed by `(x, y)`, NOT `(row, col)`.** The constructor for `SimObject` has the following form.

`SimObject(x, y, occ_array)`:
*   `x` : the x coordinate of the top left cell of the object
*   `y` : the y coordinate of the top left cell of the object
*   `occ_array` : the object's occupancy array 

Thus, an instance of `SimObject` knows its shape and its true location in the grid.

### Simulator

The `Simulator` class is what we will use to model the grid world. The constructor for `Simulator` has the following form: 


`Simulator(width, height, obj)`:
*       `width` : world width
*       `height` : world height
*       `obj` : an instance of `SimObject`, the solitary object on the board.


With the above constructor, you can initialize a `Simulator` board containing a single object. The method you will use to simulate grasps is `grasp_action(..)`:

`grasp_action(loc, gripper_dir, obj_loc)`:
*       `loc` : the index of the row or column that the gripper enters the board from
*       `gripper_dir` : which side of the board the gripper enters from. (integer, where 0 = LEFT, 1 = TOP, 2 = RIGHT, 3 = BOTTOM)
*       `obj_loc` : override the true object location. Format: `(x, y)` tuple
*        Returns a tuple of form `(stop_loc, direc, col_logic, loc)` where:

    *    `stop_loc` : `(x, y)` tuple representing the cell the gripper stopped in 
    *    `gripper_dir` : the side of the board the gripper entered from (integer, where 0 = LEFT, 1 = TOP, 2 = RIGHT, 3 = BOTTOM) (same as the argument passed in)
    *    `sensor_readings` : a 3-element list of booleans describing which sensors made contact with the object. If the middle element is `True`, the object has successfully been grasped.
    *    `loc` : the index of the row or column that the gripper entered the board from (same as the argument passed in)

Using the above method, you can simulate the gripper reaching from any direction and at any offset from the board. As specified above and as in the example in lecture, the gripper will sweep through the board until it stops, and the stop location is reported back to the user as a tuple.

### Example

In order to model this world:

<img src="world_with_obj.png" style="width: 300px;">

we would write:

`
objarr = np.array([1, 1], [0, 1])
obj = SimObject(1, 1, occ_arr=objarr)
sim = Simulator(4, 4, obj)
`

### Modeling a grasp (5 points)
Let’s get some practice using this API. Create a `Simulator` world and a `SimObj` instance to model the following example board. Then, use `grasp_action(..)` to successfully grasp the object, and return the result.

<img src="grasp_board_question1.png" style="width: 300px;">

In [21]:
def successful_grasp_of_object():
    ### BEGIN SOLUTION
    objarr = np.array([[0, 1, 1], [1, 1, 1]])
    obj = SimObject(3, 1, occ_arr=objarr)
    sim = Simulator(6, 6, obj)
    return sim.grasp_action(4, 1)
    ### END SOLUTION

In [22]:
assert(hash(str(successful_grasp_of_object())) == -8269984761785951628)
tests.test_ok()

Tests passed!!

## Helper functions

In the sections below, you will be implementing helper functions for a POMDP class. **Important**: anywhere you see `action`, `observation`, or `belief` in function descriptions below, they should follow these specifications:

### Format of POMDP components

**action**: `(loc, gripper_dir)`
- `loc` : the index of the row or column that the gripper enters the board from
- `gripper_dir` : which side of the board the gripper enters from. (integer, where 0 = LEFT, 1 = TOP, 2 = RIGHT, 3 = BOTTOM)

**observation**: `((stop_x, stop_y), [sensor_reading_1, sensor_reading_2, sensor_reading_3])`
- `(stop_x, stop_y)` : `(x, y)` tuple representing the cell the gripper stopped in
- `[sensor_reading_1, sensor_reading_2, sensor_reading_3]` : list of gripper sensor readings (booleans)

**belief**: 2D numpy array
- Each cell `(i, j)` corresponds to state `(i, j)` and contains the probability that the top left cell of the object's bounding box is located there.
 

### Initializing uniform belief over states (10 points)

First, we must write a method to get an initial belief distribution for the grasping POMDP. Uniform belief means that there is equal probability that the object is located anywhere in the simulator that it can be located without going off the board. 

Complete the method `get_uniform_belief(sim)` below, which takes a `Simulator` as its argument and returns a numpy array corresponding to a uniform belief over object locations. That is, each entry in the array has the same value, representing the probability that the top left corner of the object's bounding box is located at that cell. 

**Recall that an object's location is specified by the top left cell of its bounding box. An object must be fully within the bounds of the simulator, so the dimensions of the belief array you return will be affected by the object's dimensions (It is not necessarily the same size as the simulator).**   

In [23]:
def get_uniform_belief(sim):
    ### BEGIN SOLUTION
    import numpy as np
    width = sim.width - sim.obj.width + 1
    height = sim.height - sim.obj.height + 1
    return np.ones((height, width)) / np.sum(np.ones((height, width)))
    ### END SOLUTION

In [24]:
tests.test_uniform_belief_1(get_uniform_belief)
tests.test_uniform_belief_2(get_uniform_belief)
tests.test_uniform_belief_3(get_uniform_belief)
tests.test_ok()

Tests passed!!

### Getting an observation given a state and action (10 points)

In this domain, observations are determinstic given a state `s` and action `a`. We will take advantage of this in order to make `update_belief(..)` easier to write. 

Complete the method `obs_given_s_a(sim, s, a)` below, which takes as its arguments: a simulator `sim`; a state `(i, j)`; and an action `a`. This method returns the observation one would make under the action `a` with the object located at `(i, j)`.

**Hint**: You may find the simulator's `grasp_action(..)` useful for getting the observation.  Don't forget to translate what it returns into POMDP observation format (described in "Format of POMDP components" above).

**NOTE**: Here, `(i, j)` refers to a `(row, col)` index. You will need to translate these into `(x, y)` before passing them into `grasp_action(..)`.   

In [25]:
def obs_given_s_a(sim, s, a):
    ### BEGIN SOLUTION
    i, j = s
    # Put top-left corner of object at ith row, jth col
    top_left_x = j
    top_left_y = i
    # Run grasp action
    sim_obs = sim.grasp_action(a[0], a[1], (top_left_x, top_left_y))
    stop_loc, direc, col_logic, loc = sim_obs
    return (stop_loc, col_logic)
    ### END SOLUTION

In [26]:
tests.test_obs_given_s_a_1(obs_given_s_a)
tests.test_obs_given_s_a_2(obs_given_s_a)
tests.test_obs_given_s_a_3(obs_given_s_a)
tests.test_ok()

Tests passed!!

### Updating belief (25 points)

This method takes a belief, an action, and an observation, and applies the action and observation to transition the current belief to a new belief. It then returns the new belief.

This is akin to the update step for a Hidden Markov Model; traditionally, one must transition one’s belief based on some transition probability, and then update one’s belief using Bayes’ rule in conjunction with the observation model. 

You may use the function `prob_obs_given_s_a((i, j), a, o)` to obtain the probability of the observation `o`, given the state `(i, j)` and the action `a`. (This function uses your `obs_given_s_a(..)` implementation.)

**Hint**: for each state, the new belief is proportional to the current belief times the probability of the observation given the state and action. Don't forget that all beliefs must still sum to 1!

In [27]:
def prob_obs_given_s_a(sim, s, a, o):
    return (1.0 if (obs_given_s_a(sim, s, a) == o) else 0.0)

def update_belief(sim, b_s, a, o):
    ### BEGIN SOLUTION
    p_s = b_s

    # For each nonzero entry in p_s
    p_s_new = np.zeros(p_s.shape)
    for i in xrange(p_s.shape[0]):
        for j in xrange(p_s.shape[1]):
            if p_s[i, j] > 0:
                # Update probability based on observation
                p_s_new[i, j] = prob_obs_given_s_a(sim, (i, j), a, o) * p_s[i, j]

    p_s_new_total = np.sum(p_s_new)

    assert p_s_new_total != 0.0
    
    p_s_new = p_s_new / p_s_new_total

    return p_s_new
    ### END SOLUTION

In [28]:
tests.update_belief_test_case_1(update_belief)
tests.update_belief_test_case_2(update_belief)
tests.update_belief_test_case_3(update_belief)
tests.test_ok()

Tests passed!!

### Determining whether a belief is terminal (30 points)

What constitutes a terminal belief in our POMDP? In this case, we want the robot to have 100% confidence that it is grasping the object in the correct place. Thus, first, the belief should indicate that we are certain of the object's location. If we are not, then this is NOT a terminal belief. Second, given that the state is certain, we should make sure that we are grasping the object in the desired location.

Below, you will implement `is_terminal_belief(..)`, which takes as its arguments a simulator, a belief, an action, an observation, and a desired grasp location. It returns `True` or `False` depending on whether this belief is a terminal belief. 

The action and observation arguments represent the latest action taken and the latest observation made (the action in particular is relevant for determining whether a grasp was successful). `desired_loc` is an `(x, y)`-tuple specifying the cell on the object (in the object reference frame) that we want to grasp.  

We can make use of the method `find_obj_rel_grasp_point(..)` of `Simulator`, which gives the cell on the object (in the object reference frame) that the specified action grasped. For example, if the gripper grasped the top-left corner of the object, `(0, 0)`, would be returned. The full specification is as follows: 

`find_obj_rel_grasp_point(grasp_action_result, obj_loc)`:
- `grasp_action_result` : the return tuple from `grasp_action(..)`
- `obj_loc` : override the true object location. Format: `(x, y)` tuple

Returns `(x, y)` of where the gripper stopped, relative to the object reference frame.

In [29]:
def is_terminal_belief(sim, b_s, a, o, desired_loc):    
    ### BEGIN SOLUTION
    if a is None or o is None:
        return False   # Need to have grasped for this to be a terminal belief
    p_s = b_s
    (loc, direction) = a
    (stop_loc, sensors) = o

    certain = np.sum( (p_s == 1) + 0.0 ) == 1

    # NOTE: ONLY APPLIES if we care about where we grasp the object!
    # make sure we know 100% where the object is
    if not certain:
        return False

    # make sure latest reach was a grasp
    s1, s2, s3 = sensors
    if not s2: # we can't grasp anything without s2
        return False

    ij = np.argmax(b_s)
    assert ij < np.size(b_s)
    i = ij // len(b_s[0])
    j = ij % len(b_s[0])
    # Put top-left corner of object at ith row, jth col
    top_left_x = j
    top_left_y = i

    grasp_action_result = sim.grasp_action(a[0],a[1], (top_left_x, top_left_y))
    grasp_loc = sim.find_obj_rel_grasp_point(grasp_action_result, (top_left_x, top_left_y))
    if desired_loc == grasp_loc:
        return True

    return False
    ### END SOLUTION

In [30]:
tests.is_terminal_belief_test_case_1(is_terminal_belief)
tests.is_terminal_belief_test_case_2(is_terminal_belief)
tests.is_terminal_belief_test_case_3(is_terminal_belief)
tests.test_ok()

Tests passed!!

### Computing cost (20 points)

Finally, we must define the cost of a given node in the search tree used to solve our POMDP instances. For our purposes, we want to prioritize successfully grasping the object as quickly as possible (where "quickly" means using the fewest actions). Thus, the cost for a terminal belief will be equal to the number of actions required to reach that belief. 

If the belief is not terminal, then we must also estimate the remaining cost from that belief to a terminal belief (called the “cost-to-go”). We have provided the method `heuristic(sim, b_s)` to estimate this quantity. In that case, the cost of the belief would be the number of actions taken so far plus the heuristic cost-to-go. 

Complete the `cost(..)` function, which takes as its arguments a simulator, a belief, a list of actions taken so far, and a list of observations made so far, and a desired grasping location. It returns the cost of the belief in the search tree.

In [31]:
def cost(sim, b_s, actions, observations, desired_loc):
    ### BEGIN SOLUTION
    cost = 0

    # cost-so-far; number of actions taken
    cost += len(actions)

    # if terminal belief, then cost-so-far is the cost
    if is_terminal_belief(sim, b_s, actions[-1], observations[-1], desired_loc):
        return cost

    # if not terminal belief, need to include heuristic cost
    cost += heuristic(sim, b_s)

    return cost
    ### END SOLUTION

In [32]:
tests.cost_test_case_1(cost)
tests.cost_test_case_2(cost)
tests.cost_test_case_3(cost)
tests.test_ok()

Tests passed!!

We’re done! Now, run the following code block to synthesize your `GraspingPOMDP` class.

In [33]:
import tests

GraspingPOMDP = tests.make_grasping_pomdp_class(get_uniform_belief,
                            obs_given_s_a,
                            update_belief,
                            is_terminal_belief,
                            cost)

### Tactile Exploration Example 1: Tetris square

Great. Now, we can run the grasping POMDP and observe the results. Run the following code block, which will use your planner to repeatedly plan grasp actions until the block has been grasped. 

In [34]:
objarr =[[1,0],
         [1,0],
         [1,1]]
objarr = np.transpose(objarr)
obj = SimObject(4, 5, objarr)
sim = Simulator(10, 10, obj)
desired_loc = (0, 0)

# init
gp = GraspingPOMDP(sim, desired_loc)
grid = graphics.Grid(sim.width, sim.height)
beliefs = []
actions = []
directions = ['LEFT','TOP','RIGHT','BOTTOM']
max_itr = 50
b_s = gp.get_uniform_belief()
a, o = None, None
beliefs = [b_s]

while max_itr > 0 and not gp.is_terminal_belief(b_s, a, o):
    print 'About to solve...'
    score, a = gp.solve(b_s, depth=1, max_actions=None)
    print 'score: %s action: %s (%s)' % (str(score), str(a),str(directions[a[1]]))
    actions.append(a)
    sim_obs = sim.grasp_action(a[0],a[1])
    o = gp.sim_obs_to_pomdp_obs(sim_obs)
    print 'obs: %s' % str(o)
    b_s = gp.update_belief(b_s, a, o)
    beliefs.append(b_s)
    print 'is_terminal_belief(b_s): %s' % str(gp.is_terminal_belief(b_s,a,o))
    print
    max_itr -= 1
actions.append(actions[-1])

print 'Terminated after %d actions.' % (50-max_itr)

grid.visualize_grasp_sequences(sim, beliefs, actions)


About to solve...
score: -134.875 action: (5, 0) (LEFT)
obs: (3, [False, False, True])
is_terminal_belief(b_s): False

About to solve...
score: -1.0 action: (4, 1) (TOP)
obs: (5, [False, True, False])
is_terminal_belief(b_s): True

Terminated after 2 actions.


The next cell will animate the grasping actions. The shaded teal cells in the grid show a heatmap of the belief of where the object is. You can see it evolve over time as the belief becomes more certain. 

In [35]:
# Now let's animate it (give it a second to load):
grid.play()

### Tactile Exploration Example 2: Pole (10 points)

Next, run the following code to see how your POMDP planner handles the challenge of grasping an object which is difficult to localize and which requires tactile exploration actions in the worst case in order to successfully grasp it.

In [36]:
import tests

objarr =[[0,1],
         [1,1],
         [0,1],
         [1,1],
         [0,1],
         [1,1],
         [0,1],
         [1,1],
         [0,1]]
objarr = np.transpose(objarr)
obj = SimObject(3, 3, objarr)
sim = Simulator(8, 18, obj)
desired_loc = (0,3)

#init
gp = GraspingPOMDP(sim, desired_loc)
grid = graphics.Grid(sim.width, sim.height)
beliefs = []
actions = []
directions = ['LEFT','TOP','RIGHT','BOTTOM']
max_itr = 50
b_s = gp.get_uniform_belief()
a, o = None, None

beliefs.append(b_s)
while max_itr > 0 and not gp.is_terminal_belief(b_s, a, o):
    print 'About to solve...'
    score, a = gp.solve(b_s, depth=1)
    print 'score: %s action: %s (%s)' % (str(score), str(a),str(directions[a[1]]))
    actions.append(a)
    sim_obs = sim.grasp_action(a[0],a[1])
    o = gp.sim_obs_to_pomdp_obs(sim_obs)
    print 'obs: %s' % str(o)
    b_s = gp.update_belief(b_s, a, o)
    beliefs.append(b_s)
    print 'is_terminal_belief(b_s): %s' % str(gp.is_terminal_belief(b_s,a,o))
    max_itr -= 1
actions.append(actions[-1])

print 'Terminated after %d actions.' % (50-max_itr)

grid.visualize_grasp_sequences(sim, beliefs, actions)

About to solve...
score: -12.8857142857 action: (8, 0) (LEFT)
obs: (3, [True, True, True])
is_terminal_belief(b_s): False
About to solve...
score: -2.0 action: (2, 1) (TOP)
obs: (3, [False, False, True])
is_terminal_belief(b_s): False
About to solve...
score: -1.0 action: (6, 0) (LEFT)
obs: (3, [True, True, True])
is_terminal_belief(b_s): True
Terminated after 3 actions.


In [37]:
grid.play()

Now, let's analyze the results:

- How many actions were required for your POMDP to localize and grasp the object? 
- Why was the first successful grasp insufficient? 
- Why would repeatedly reaching for a maximum-likelihood estimate of the grasp location be insufficient? (Hint: why are exploratory actions (which don't directly aim to grasp the object) helpful?)


* It took 3 actions.
* One grasp was insufficient as we required more grasps than that in order to successfully localize the object, and localization was particularly important here as we wanted to ensure that we had successfully grasped the object in a particular location.
* Repeatedly reaching for the maximum likelihood estimate of the grasp location would not have reduced our uncertainty about where this object was located, and so we never would have reached a terminal belief (instead we have to take exploratory actions).


Done! Please run the validator and submit this notebook!